<a href="https://colab.research.google.com/github/gianca1990/pdf_extraction/blob/main/berami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google import genai
from google.genai import types
import pathlib
import httpx
import csv

In [78]:
# Initialize the Google GenAI client with the provided API key
client = genai.Client(api_key="")

In [42]:
# Filepath to the local PDF file
filepath = pathlib.Path("/content/EIGENE_DATEIEN/TO_LOAD.pdf")

In [43]:
# Read the PDF file as bytes
pdf_bytes = filepath.read_bytes()

## Header

In [76]:
# Prompt for extracting specific data from the PDF
prompt = """
Extract the following information from the attached PDF and return it as CSV:
Datum:;
Berater*in:;
Extract the content of the fields that are marked with a checked checkbox (✔ or ☑) between 'Telefonberatung:' and 'E-Mail-/Präsenz-/Zoom-Beratung:'.

Output only the marked text. Format the output as CSV with the headers:
Datum;
Berater*in;
Beratung.

Use semicolons as separators. If a piece of information is missing, leave the field blank. Only extract the dataset in the CSV format. Do not include any additional information, explanations, or text."""

In [79]:
# Send the PDF and prompt to Google GenAI for content generation
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])

In [58]:
# Print the response text for debugging or confirmation
print(response.text)

```csv
Datum;Berater*in;Beratung
300724;DV;Die ratsuchende Person hat zugestimmt, dass ihre Daten gespeichert und verarbeitet werden dürfen (Uhrzeit)
```



## Persönliche Daten Ratsuchend*er (RS*)

In [66]:
# IMPORTANT: The extraction of the information for "Geschlecht:" seems to be difficult.
# Prompt for extracting specific data from the PDF
prompt = """
Extract the following information from the attached PDF and return it as CSV:
Extract the content of the fields that are marked with a checked checkbox (✔ or ☑) between 'Person mit ausl. Abschl. selbst' and 'soziales Umfeld';
Name:;
Vorname:;
E-Mail:;
Telefonnummer:;
Extract the content of the fields that are marked with a checked checkbox (✔ or ☑) next to the heading "Geschlecht:". The options are 'm', 'w', 'divers', and 'keine Angabe';
Extract the content of the fields that are marked with a checked checkbox (✔ or ☑) next to the heading "Wohnsitz in Deutschland?". The options are 'ja, in (Wohnort):' and 'nein, in (Land)';
If the option 'ja, in (Wohnort):' is marked, extract the content of the field next to it. If the option 'nein, in (Land)' is marked, extract the content of the field next to it.;
Einreisejahr:;
Geburtsjahr:;
Geburtsland:;
Nationalität 1:;
ggf. Nationalität 2.

Output only the marked text. Format the output as CSV with the headers:
Wer nimmt die Beratung wahr?;
Name;
Vorname;
E-Mail;
Telefonnummer;
Geschlecht;
Wohnsitz in Deutschland?;
Wohnsitz;
Einreisejahr;
Geburtsjahr;
Geburtsland;
Nationalität 1;
ggf. Nationalität 2.

Use semicolons as separators. If a piece of information is missing, leave the field blank. Only extract the dataset in the CSV format. Do not include any additional information, explanations, or text."""

In [67]:
# Send the PDF and prompt to Google GenAI for content generation
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])

In [68]:
# Print the response text for debugging or confirmation
print(response.text)

Wer nimmt die Beratung wahr?;Name;Vorname;E-Mail;Telefonnummer;Geschlecht;Wohnsitz in Deutschland?;Wohnsitz;Einreisejahr;Geburtsjahr;Geburtsland;Nationalität 1;ggf. Nationalität 2
Person mit ausl. Abschl. selbst;Mustermann;Max;arielaw9@gmail.com;016093455298;m;ja, in (Wohnort):;Frankfurt;2023;1963;Israel;israelisch;




## Berufliche Situation und Aufenthaltsstatus

In [72]:
# IMPORTANT: Each field extraction must be provided with the addition "If the checkbox is not checked, leave the field blank.".
# Prompt for extracting specific data from the PDF
prompt = """
Extract the following information from the attached PDF and return it as CSV:
Extract the content of the fields that are marked with a checked checkbox (✔ or ☑) next to the heading "Ist der/die RS erwerbstätig". The options are 'ja, im Ausland', 'nein', 'keine Angabe', 'ja, in Deutschland' and 'nein, jedoch im Aus-/Weiterbildung/Qualifizierung';
Extract the content of the fields that are marked with a checked checkbox (✔ or ☑) next to the heading "Art der Erwerbstätigkeit". The options are 'beitragspflichtig beschäftigt', 'geringfügig beschäftigt' and 'selbstständig'. If the checkbox is not checked, leave the field blank.

Output only the marked text. Format the output as CSV with the headers:
Ist der/die RS erwerbstätig;
Art der Erwerbstätigkeit.

Use semicolons as separators. If a piece of information is missing, leave the field blank. Only extract the dataset in the CSV format. Do not include any additional information, explanations, or text."""

In [73]:
# Send the PDF and prompt to Google GenAI for content generation
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])

In [74]:
# Print the response text for debugging or confirmation
print(response.text)

```csv
Ist der/die RS erwerbstätig;Art der Erwerbstätigkeit
nein;
```



## Bezug & Status

In [92]:
# IMPORTANT: Each field extraction must be provided with the addition "If the checkbox is not checked, leave the field blank.".
# Prompt for extracting specific data from the PDF
prompt = """
Extract the following information from the attached PDF and return it as CSV:
Extract the content of the fields that are marked with a checked checkbox. But only between he following options: 'ohne Leistungsbezug', 'ohne Leistungsbezug, jedoch arbeitssuchend gemeldet', 'mit (ergänzendem) SGB II-Leistungsbezug', 'mit SGB III-Leistungsbezug', 'mit SGB III- und SGB II-Leistungsbezug', 'mit (ergänzendem) Asylbewerberleistungsbezug' and 'keine Angabe'. If the checkbox is not checked, leave the field blank.;
Extract the content of the fields that are marked with a checked checkbox.
But only between he following options:
'Aufenthaltserlaubnis zur Arbeitsplatzsuche (§20 AufenthG)',
'Aufenthaltserlaubnis für in anderen Mitgliedsstaaten der Europäischen Unionlangfristig Aufenthaltsberechtigte (§38a AufenthG)',
'Aufenthaltsgestattung (§55 Abs. 1 AsylVfG)',
'Aufenthalt aus familiären Gründen (§27-36 AufenthG)',
'Aufenthalt aus völkerrechtlichen, humanitären oder politischen Gründen(22-26, 104a, 104b AufenthG)',
'Aufenthalt zum Zweck der Ausbildung (§16a und 17 AufenthG)',
'Aufenthalt zum Zweck der Erwerbstätigkeit (§18, 18a, 20, 21 AufenthG)',
'Aufenthalt zum Zweck einer Anpassungsqualifizierung oder einer Kenntnisprüfung16d AufenthG)',
'Blaue Karte EU (§18b, Abs. 2 AufenthG)',
'Duldung (§60a, 60b, 60c, 60d AufenthG)',
'Niederlassungserlaubnis (§9 AufenthG)',
'Staatsbürger-/in EU/EWR/Schweiz oder Freizügigkeit 2-5, §§12-13, §15 FreizügG/EU, §28 AufenthV)☐Visum (§6 AufenthG)' and
'kein Aufenthaltstitel, da Wohnsitz im Ausland'
If the checkbox is not checked, leave the field blank.

Output only the marked text. Format the output as CSV with the headers:
Leistungsbezug;
Art der Erwerbstätigkeit.

Use semicolons as separators. If a piece of information is missing, leave the field blank. Only extract the dataset in the CSV format. Do not include any additional information, explanations, or text."""

In [96]:
# Send the PDF and prompt to Google GenAI for content generation
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])

In [97]:
# Print the response text for debugging or confirmation
print(response.text)

```csv
Leistungsbezug;Art der Erwerbstätigkeit
;beitragspflichtig beschäftigt
```



In [9]:
# Define the output CSV filename
filename = "/content/EIGENE_DATEIEN/DATA.csv"

In [115]:
import fitz

def extract_checked_checkboxes(pdf_path):
    doc = fitz.open(pdf_path)
    for page in doc:
        for widget in page.widgets():  # Alle interaktiven Felder durchgehen
            if widget.field_type == 2:  # 2 = Checkbox
                value = widget.field_value  # Wert der Checkbox abrufen
                if value == "On":  # Nur Checkboxen mit "On" ausgeben
                    print(f"Feldname: '{widget.field_name}' | Wert: '{value}'")

pdf_path = "/content/EIGENE_DATEIEN/TO_LOAD.pdf"
extract_checked_checkboxes(pdf_path)

Feldname: 'Telefonberatung' | Wert: 'On'
Feldname: 'schriftliche Einwilligungserklärung liegt vor' | Wert: 'On'
